In [2]:
from binance.client import Client
from binance.enums import *
from time import time
import pickle as pickle
from datetime import datetime
import pandas as pd
from pathlib import Path
from datetime import datetime
import json
import math
import ast
from time import time
import pickle


from typing import List, Dict


downloads_path = str(Path.home() / "Downloads")
documents_path = str(Path.home() / "Documents")

In [7]:
# Extract good long patterns
def load_dict_from_pickle(filename):
    with open(filename, 'rb') as file:
        dictionary = pickle.load(file)
    print(f'Reading {filename}')
    return dictionary

long_sorted_by_pattern_occurrence = load_dict_from_pickle(f'{downloads_path}/sorted_by_pattern_occurrence.pkl')
long_patterns = []
for val in long_sorted_by_pattern_occurrence:
    pattern_occurrence = val['pattern_occurrence']
    if pattern_occurrence > 500:
        long_patterns.append(val['tst_chunk'])

Reading C:\Users\HK\Downloads/sorted_by_pattern_occurrence.pkl


In [10]:
# Download unseen historical data

def get_client():
    fn = f'{documents_path}\\key\\binance-key.pickle'
    with open(fn, 'rb') as handle:
        k = pickle.load(handle)
    return Client(k['API_KEY'], k['API_SECRET'])

def get_config(file_path):
    with open(file_path, 'r') as file:
        config = json.load(file)
    return config

def download_hd(start_timestamp, end_timestamp, candlestick): 
    client = get_client()
    data = []
    tot = (end_timestamp - start_timestamp)/(900*500)
    cntr = 0
    for current_sts in range(start_timestamp, end_timestamp+1, 900*500):
        next_ets = current_sts + 900*500 if (current_sts + 900*500) < end_timestamp else end_timestamp
        print(current_sts, next_ets, f'100% completed') if next_ets == end_timestamp else print(current_sts, next_ets, f'{round(cntr*100/tot, 1)}% completed')
        cntr += 1
        
        klines = client.futures_historical_klines('BTCUSDT', candlestick, current_sts*1000, next_ets*1000, limit=500)
        
        for kline in klines:
            timestamp = kline[0]/1000
            open_price = float(kline[1])
            high_price = float(kline[2])
            low_price = float(kline[3])
            close_price = float(kline[4])
            volume = float(kline[5])

            data.append([timestamp, open_price, high_price, low_price, close_price, volume])

    df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
    hd_dl_fn = f'{downloads_path}/unseen_hd_raw.csv'
    df.to_csv(hd_dl_fn, index=False)
    print('Historical Data Exported!\t', hd_dl_fn)
    return df


config = get_config('config.json')
candlestick = config['candlestick']
chunk_size = config['chunk_size']
roi_threshold = config['roi_threshold']
sm_threshold = config['sm_threshold']

start_timestamp = 1719721304 #  Sunday, June 30, 2024 8:21:44 AM GMT+04:00
end_timestamp = 1722081600 #  July 27, 2024 4:00:00 PM GMT+04:00

unseen_hd = download_hd(start_timestamp, end_timestamp, candlestick)


1719721304 1720171304 0.0% completed
1720171304 1720621304 19.1% completed
1720621304 1721071304 38.1% completed
1721071304 1721521304 57.2% completed
1721521304 1721971304 76.3% completed
1721971304 1722081600 100% completed
Historical Data Exported!	 C:\Users\HK\Downloads/unseen_hd_raw.csv


In [15]:

def get_pc(hd_df):
    close_price = list(hd_df['close'])
    ts = list(hd_df['time'])
    pc = []
    for idx, val in enumerate(close_price):
        if idx==0:
            pc.append({'time': ts[0], 'close_pc': 0})
        else:
            previous_price = close_price[idx-1]
            current_price = val
            pc_val = (round((current_price*100/previous_price)-100, 4))
            pc.append({'time': ts[idx], 'close_pc': pc_val})
    pc_df = pd.DataFrame(pc)
    return pc_df

def get_chunks(qt_df):
    lst = list(qt_df['close_pc'])
    r=[]
    tot = len(lst)
    for idx, val in enumerate(lst):
        if idx < chunk_size:
            r.append([0]*chunk_size)
        else:
            r.append(lst[idx-chunk_size:idx])
    qt_df['ref_chunks'] = r
    qt_df.to_csv(f'{downloads_path}/backtest_chunks.csv', index=False)
    print('backtest_chunks Chunks Exported!', f'{downloads_path}/backtest_chunks.csv')
    return qt_df

hd_dl_fn = f'{downloads_path}/unseen_hd_raw.csv'
unseen_hd = pd.read_csv(hd_dl_fn)
pc_df = get_pc(unseen_hd)
backtest_chunks = get_chunks(pc_df)


backtest_chunks Chunks Exported! C:\Users\HK\Downloads/backtest_chunks.csv


In [47]:

def all_zeros(lst):
    return all(x == 0 for x in lst)

def format_elapsed_time(seconds):
    days = seconds // (24 * 3600)
    seconds %= 24 * 3600
    hours = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60

    return f"{days:03}d:{hours:02}h:{minutes:02}m:{seconds:02}s"

def chunk_diff(long_db_chunk, tst_lst, tst_df):
    ts = list(tst_df['time'])
    test_chunks = list(tst_df['ref_chunks'])
    cntr = 0
    chunk_size = len(tst_lst[0])
    r = []
    tot = len(tst_lst)
    start_ts = int(time())
    log_msg = []
    enter_long_pos = []
    for tst_lst_idx,tst_val in enumerate(tst_lst):
        tst_lst_idx += 1
        pattern_occurrence = 0
        long_rois_lst = []
        short_rois_lst = []
        avg_long_roi = 0
        avg_short_roi = 0
        avg_sm = 0
        sm_lst = []
        qal_long_db_chunk_vals = []
        if(tst_lst_idx%10 == 0) and tst_lst_idx != 0:
            completed = round(tst_lst_idx*100/tot, 1)
            remaining = round(100-completed)
            cur_ts = time()
            ts_diff = int(cur_ts - start_ts)
            estimated_tm_to_complete = round(int(ts_diff*(tot-tst_lst_idx)/tst_lst_idx), 1)
            print(f"\rElapsed Time: {format_elapsed_time(ts_diff)} \t Completed: {completed}% \tRemaining: {remaining}% \tETA: {format_elapsed_time(estimated_tm_to_complete)}", end="")
        for ref_idx,ref_val in enumerate(long_db_chunk):
            diff = []
            if(all_zeros(ref_val)) or (all_zeros(tst_val) or (tst_val == ref_val)):
                continue
            else:                
                for k in range(chunk_size):
                    diff.append(abs(tst_val[k] - ref_val[k]))
                cntr += 1
                sm = round(sum(diff), 2)
                if (sm <= sm_threshold):
                    # ts_val = ts[tst_lst_idx]
                    if tst_val in enter_long_pos:
                        pass
                    else:
                        enter_long_pos.append(tst_val)
                    qal_long_db_chunk_vals.append(ref_val)
                    sm_lst.append(sm)
                    pattern_occurrence += 1
                    # sm_lst.append({'sm': sm, 'pattern_id': pattern_id, 'long_roi': long_roi, 'short_roi': short_roi, 'ref': ref_val, 'tst': tst_val})

    # enter_long_pos_df = pd.DataFrame(enter_long_pos)
    # print('\nSaving enter_long_pos_df...')
    # enter_long_pos_df.to_csv(f'{downloads_path}/enter_long_pos_df.csv', index=False)
    return enter_long_pos

tst_lst = list(backtest_chunks['ref_chunks'])
enter_long_poss = chunk_diff(long_patterns, tst_lst, pc_df)


Elapsed Time: 000d:00h:00m:35s 	 Completed: 99.9% 	Remaining: 0% 	ETA: 000d:00h:00m:00ss

In [53]:
print(enter_long_poss[1])

[-0.1212, -0.0402, -0.1469, -0.1554, 0.0781, -0.0503, 0.1552, 0.0309, 0.1614, -0.1075]
